In [1]:
import os
import gc 
from glob import glob
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import lightgbm as lgb
import torch
import torch.nn as nn

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier, Pool
import xgboost
from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
ROOT            = Path("/kaggle/input/home-credit-risk-model-train-test")
SCHEMA_PATH            = Path("/kaggle/input/schema-home-credit-risk-data")

In [3]:
def set_data_types(df, schema):
    
    for col, dtype in schema.items():
        if dtype == 'category':
            dtype = 'object'
        df[col] = df[col].astype(dtype)
    return df
    

In [4]:
X = pl.read_csv(ROOT/'model_abt_train.csv').to_pandas()
schema = pd.read_csv(SCHEMA_PATH / "data_schema.csv", names = ['Columns', 'dtype'])
schema = schema[schema['Columns'].isin(X.columns)]
schema = schema.set_index('Columns')['dtype'].to_dict()
X = set_data_types(X, schema)
del schema

In [5]:
id_val = X[['case_id', 'WEEK_NUM']]
y = X['target']
cols = X.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1).columns
X = X.drop(['case_id', 'WEEK_NUM', 'target', 'decision_month', 'decision_weekday'], axis = 1)

In [6]:
stats = X.describe().T
stats['perc_nulls'] = stats['count']/len(X)
drop = stats[stats['perc_nulls']>0.95].index
print(drop)
X = X.drop(drop, axis = 1)
del stats
del drop
gc.collect()

Index(['annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L',
       'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L',
       'applicationscnt_629L', 'applicationscnt_867L',
       'clientscnt12m_3712952L', 'clientscnt_100L', 'clientscnt_1022L',
       'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L',
       'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L',
       'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L',
       'clientscnt_946L', 'credamount_770A', 'currdebtcredtyperange_828A',
       'disbursedcredamount_1113A', 'downpmt_116A', 'homephncnt_628L',
       'mobilephncnt_593L', 'numactivecreds_622L',
       'numactivecredschannel_414L', 'numactiverelcontr_750L',
       'numcontrs3months_479L', 'numinstls_657L', 'numnotactivated_1143L',
       'numpmtchanneldd_318L', 'numrejects9m_859L', 'pmtnum_254L',
       'sellerplacecnt_915L', 'sellerplacescnt_216L', 'totalsettled_863A',
       'first_birth_259D', 'mainoccupationinc_384A_m

0

In [7]:
from collections import Counter
Counter(X.dtypes)

Counter({dtype('float64'): 327, dtype('O'): 81, dtype('bool'): 1})

In [8]:
cat_cols = [x for x, dtypes in X.dtypes.items() if dtypes == 'O']
num_cols = [x for x, dtypes in X.dtypes.items() if dtypes in ['float', 'int']]
X[cat_cols] = X[cat_cols].astype(str)

In [9]:
score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}
scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
scores

,model_name,model,params,acc,precision,recall,f1,roc_auc,pr_auc,lift


In [10]:
class Model_Utils:
    @staticmethod
    def model_evals(y_true, y_proba, cutoff = 0.5):
        from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, average_precision_score, roc_auc_score
        """
        Returns model evaluation metrics for a binary classification model

        Parameters:
        -----------
            y_true: int (0,1) 
                Actual binary labels

            y_proba: float (between 0 and 1)
                Probability scores output of model 

        Returns:
        --------
            result: dict
                Dictionary of metrics and their results based on the input
                    - event rate (% predicted 1's)
                    - accuracy
                    - roc_auc
                    - pr_auc
                    - recall
                    - precision
                    - f1
                    - lift
        """

        y_pred = (y_proba > cutoff).astype(int)

        event_rate = y_pred.mean()

        accuracy = accuracy_score(y_true, y_pred)

        roc_auc = roc_auc_score(y_true, y_proba)

        pr_auc = average_precision_score(y_true, y_proba)

        recall = recall_score(y_true, y_pred)

        precision = precision_score(y_true, y_pred)

        f1 = f1_score(y_true, y_pred)

        lift = recall / event_rate

        return {'event_rate': event_rate,
                'acc': accuracy, 
                'precision': precision, 
                'recall': recall, 
                'f1': f1, 
                'roc_auc': roc_auc,
                'pr_auc': pr_auc, 
                'lift': lift}
    
    @staticmethod
    def DumbClassifier(y_true):
        import numpy as np
        y_pred = np.zeros(len(y_true))
        return y_pred
    
    @staticmethod
    def RandomChanceClassifier(y_true):
        import numpy as np
        y_pred = Model_Utils.DumbClassifier(y_true)
        event = y_true.sum()
        ind = np.random.randint(0, len(y_true), size=event)
        y_pred[ind] = 1
        return y_pred
    
    @staticmethod
    def save_model_results(results, schema, filepath):
        if not os.path.exists(filepath):
            score_schema = {'model_name': 'str', 
                        'model': 'str',
                        'params': 'str',
                        'acc': 'float',
                        'precision': 'float',
                        'recall': 'float',
                        'f1_score': 'float',
                        'roc_auc': 'float',
                        'pr_auc': 'float',
                        'lift': 'float'}
            scores = pd.DataFrame(columns = score_schema.keys()).astype(score_schema)
        else:
            scores = pd.read_csv(filepath)
        row = []
        
        for key in schema.keys():
            row.append(results[key])
        
        scores.loc[len(scores)] = row
        scores.to_csv(filepath, index = False)
        
    @staticmethod
    def LightGBMClassifier_CV(X, y, cat_cols, cv = 5, group = None, params = None):
        import lightgbm
        from sklearn.model_selection import StratifiedGroupKFold
        
        cv = StratifiedGroupKFold(n_splits=cv, shuffle = True, random_state = 42)
        
        if params == None:
            params = {'random_state': 42
                     ,'objective': 'binary'
                     ,'verbose': -1
                     ,'n_jobs': -1}
        
        scores = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        split = 1
        
        for train_ind, valid_ind in cv.split(X, y, groups=group):
            X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
            X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]

            X_train[cat_cols] = X_train[cat_cols].astype("category")
            X_valid[cat_cols] = X_valid[cat_cols].astype("category")
            
            lgb = lightgbm.LGBMClassifier(**params)
            lgb.fit(X_train, y_train)

            y_proba = lgb.predict_proba(X_valid)[:, 1]
            results = Model_Utils.model_evals(y_valid, y_proba)
            print(f'LightGBM, Val CV{split}: {results}"')
            
            split += 1
            
            for key in results.keys():
                if key in scores.keys():
                    scores[key].append(results[key])
        
        for key in results.keys():
            if key in scores.keys():
                scores[key] = np.array(scores[key]).mean()
            
        print(f'LightGBM, Average CV: {scores}"')
        return lgb, scores

    @staticmethod
    def CatBoostClassifier_CV(X, y, cat_cols, cv = 5, group = None, params = None):
        import catboost
        from sklearn.model_selection import StratifiedGroupKFold
        
        cv = StratifiedGroupKFold(n_splits=cv, shuffle = True, random_state = 42)
        
        if params == None:
             params = {'boosting_type' : "Plain",
                      'eval_metric': 'PRAUC',
                      'random_seed': 42,
                      'learning_rate': 0.05,
                      'use_best_model': True,
                      'iterations': 1000}
        
        scores = {'acc': [],
                        'precision': [],
                        'recall': [],
                        'f1': [],
                        'roc_auc': [],
                        'pr_auc': [],
                        'lift': []}

        split = 1
        
        for train_ind, valid_ind in cv.split(X, y, groups=group):
            X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
            X_valid, y_valid = X.iloc[valid_ind], y.iloc[valid_ind]
            
            
            clf = CatBoostClassifier(**params)
        
            train_pool = Pool(X_train, y_train, cat_features=cat_cols)
            val_pool = Pool(X_valid, y_valid, cat_features=cat_cols)
            
            clf.fit(train_pool, eval_set=val_pool, verbose=False)
            
            y_proba = clf.predict_proba(X_valid)[:, 1]
            results = Model_Utils.model_evals(y_valid, y_proba)
            print(f'CatBoost, Val CV{split}: {results}"')
            
            split += 1
            
            for key in results.keys():
                if key in scores.keys():
                    scores[key].append(results[key])
        
        for key in results.keys():
            if key in scores.keys():
                scores[key] = np.array(scores[key]).mean()
            
        print(f'CatBoost, Average CV: {scores}"')
        return clf, scores

    
    @staticmethod
    def train_test_split(X, y, test_size = 0.2):
        from sklearn.model_selection import train_test_split
        import polars as pl
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
        pl.from_pandas(pd.concat([y_train, X_train], axis = 1)).write_csv('model_abt_train.csv')
        del X_train
        del y_train
        gc.collect()
        
        pl.from_pandas(pd.concat([y_test, X_test], axis = 1)).write_csv('model_abt_test.csv')
        del X_test
        del y_test
        gc.collect()

In [11]:
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
#from utils.model_helpers import model_evals
import xgboost
import lightgbm
from time import time
from sklearn.model_selection import cross_validate, StratifiedKFold
import numpy as np

space = {
    'iterations': hp.quniform('iterations', 50, 300, 50),
    'max_ctr_complexity': hp.quniform('max_ctr_complexity', 1, 10, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.5),
    'depth': hp.quniform('depth', 3, 10, 1),
    'l2_leaf_reg': hp.quniform('l2_leaf_reg', 1, 9, 1),
    'bagging_temperature': hp.uniform('bagging_temperature', 0.5, 4),
    'border_count': hp.quniform('border_count', 32, 128, 32),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.5, 1.0),
    'boosting_type': hp.choice('boosting_type', ['Ordered', 'Plain']),
    'random_state': 42,
    'one_hot_max_size': 50,
    'verbose': -1
}


def objective(space):
    start_time = time()

    print(f"Parameters: {space}")
    space['iterations'] = int(space['iterations'])
    space['depth'] = int(space['depth'])
    space['border_count'] = int(space['border_count'])
    
    model_name = f'CatBoost_Tuned{len(scores)}'
    model_type = 'CatBoost'
    params = space
    clf, dict_results = Model_Utils.CatBoostClassifier_CV(X, y, cat_cols, group = id_val['WEEK_NUM'], params = space)  

    elapsed_time = time() - start_time
    
    row = [model_name, model_type, params]
    
    for key in dict_results.keys():
        row.append(dict_results[key])

    
    print(row)
    scores.loc[len(scores)] = row
    scores.to_csv('CatBoost_Tuned_Train.csv')

    return {'loss': -dict_results['pr_auc'], 'status': STATUS_OK, 'model': clf, 'elapsed_time': elapsed_time}

In [12]:
trials = Trials()
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 250, trials = trials)

  0%|          | 0/250 [00:00<?, ?trial/s, best loss=?]

Parameters: {'bagging_temperature': 1.283629106405163, 'boosting_type': 'Plain', 'border_count': 64.0, 'colsample_bylevel': 0.7390616780017791, 'depth': 9.0, 'iterations': 250.0, 'l2_leaf_reg': 4.0, 'learning_rate': 0.10880802542314588, 'max_ctr_complexity': 2.0, 'one_hot_max_size': 50, 'random_state': 42, 'verbose': -1}
  0%|          | 0/250 [00:00<?, ?trial/s, best loss=?]

CatBoost, Val CV1: {'event_rate': 0.0007410285838220809, 'acc': 0.9695093848559061, 'precision': 0.49390243902439024, 'recall': 0.012007115327601542, 'f1': 0.023444283646888567, 'roc_auc': 0.855956314868169, 'pr_auc': 0.19092028839312422, 'lift': 16.203309278126877}"
  0%|          | 0/250 [06:56<?, ?trial/s, best loss=?]

CatBoost, Val CV2: {'event_rate': 0.000601631686446426, 'acc': 0.969834110600597, 'precision': 0.5286624203821656, 'recall': 0.010531658418982362, 'f1': 0.02065190345857178, 'roc_auc': 0.8416256854033659, 'pr_auc': 0.16450103190418341, 'lift': 17.505159146766754}"
  0%|          | 0/250 [13:43<?, ?trial/s, best loss=?]

CatBoost, Val CV3: {'event_rate': 0.0008408864884095117, 'acc': 0.9652851332885939, 'precision': 0.5432692307692307, 'recall': 0.013131900058105753, 'f1': 0.025643935095881087, 'roc_auc': 0.8359471450724557, 'pr_auc': 0.18701193489884607, 'lift': 15.616733339292898}"
  0%|          | 0/250 [20:31<?, ?trial/s, best loss=?]

...

['CatBoost_Tuned17', 'CatBoost', {'bagging_temperature': 0.9608528668333098, 'boosting_type': 'Plain', 'border_count': 32, 'colsample_bylevel': 0.9048483317445397, 'depth': 7, 'iterations': 50, 'l2_leaf_reg': 4.0, 'learning_rate': 0.3369638224019465, 'max_ctr_complexity': 6.0, 'one_hot_max_size': 50, 'random_state': 42, 'verbose': -1}, 0.968491040329773, 0.46666982772135235, 0.011614037941675363, 0.022657382453724895, 0.8350644547996449, 0.1645395709371067, 14.825698097622118]
  7%|▋         | 17/250 [10:36:53<147:03:33, 2272.16s/trial, best loss: -0.1774456729225357]

  7%|▋         | 18/250 [10:36:53<110:27:52, 1714.10s/trial, best loss: -0.1774456729225357]

Parameters: {'bagging_temperature': 2.634000139322858, 'boosting_type': 'Ordered', 'border_count': 128.0, 'colsample_bylevel': 0.5677233864090432, 'depth': 8.0, 'iterations': 200.0, 'l2_leaf_reg': 3.0, 'learning_rate': 0.30534905574542115, 'max_ctr_complexity': 6.0, 'one_hot_max_size': 50, 'random_state': 42, 'verbose': -1}
  7%|▋         | 18/250 [10:36:53<110:27:52, 1714.10s/trial, best loss: -0.1774456729225357]

CatBoost, Val CV1: {'event_rate': 0.001278726153790542, 'acc': 0.9694506447852372, 'precision': 0.4734982332155477, 'recall': 0.019863622887637118, 'f1': 0.038127756437615595, 'roc_auc': 0.8528389054272281, 'pr_auc': 0.18215436234388843, 'lift': 15.533914613973574}"
  7%|▋         | 18/250 [10:58:38<110:27:52, 1714.10s/trial, best loss: -0.1774456729225357]

CatBoost, Val CV2: {'event_rate': 0.0013757055760144391, 'acc': 0.9696041876631016, 'precision': 0.42896935933147634, 'recall': 0.019540667427991373, 'f1': 0.03737864077669903, 'roc_auc': 0.8391240994807, 'pr_auc': 0.160926358514733, 'lift': 14.20410571032408}"
  7%|▋         | 18/250 [11:19:58<110:27:52, 1714.10s/trial, best loss: -0.1774456729225357]